In [13]:
#####################################################################
#
#    CREATED : May 3, 2021
#
#
#    NAME: 5)GalaxyPriors.npy
#
#
#    FORMER NAME : P)TheFinalBackground.ipynb
#
#    WHAT : This will help you get the priors for the galaxy background.
#                --> Gets you bgAverage & bgCovMatrix
#
#           And then WE  WILL CALCULATE THE BIAS OF THE LIKELIHOOD FUNCTION
#
#
#    DATA:
#
#            5)data/chain_3x2pt_lcdm_SR_maglim.txt
#
#
#####################################################################

# output

   - [ ] '5)dataOut/bgAverage.npy'                
   - [ ] '5)dataOut/bgCovMatrix.npy 

In [14]:
from __future__ import division
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy
import matplotlib.pyplot as plt
import math
import sys, platform, os
import re
import time
import pprint
from numpy import genfromtxt
import importlib
plt.rcParams['figure.figsize']=(18,18)
%matplotlib inline
print(os.getcwd())
print(platform.python_version(),'python version')
import matplotlib.cm as cmap

#Astropy
from astropy import units as u
from astropy.coordinates import Angle
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy.utils.data import download_file
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy.visualization import astropy_mpl_style
#Astropy cosmology
from astropy.cosmology import WMAP9 as cosmo

#Scipy
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from scipy import signal


from chainconsumer import ChainConsumer

#treecorr
import treecorr

#pandas
import pandas as pd

# fitsio
import fitsio

#healpy
import healpy as hp 

#Linear Al
from numpy.linalg import inv

#h5py
import h5py

#CAMB
import camb
from camb import model, initialpower,get_matter_power_interpolator

# Print
np.set_printoptions(threshold=50)

#corner plot
import corner

#emcee
import emcee

# Let's get the pretty colors
cmap = plt.get_cmap('spring')

# This is the main Library where all the definitions are kept 
F =  __import__('1)FLibrary')
J =  __import__('2)JLibrary')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/tzegoh_1/Desktop/-1_0th_Project_REDO
3.12.7 python version


# CONSTRUCT PRIORS 

We should switch over to using the actual prior on the galaxy bias parameters that
comes from other DES analysis.  To construct that prior, you can use the file below. 

The first line of the file explains the columns. 
After a bunch of header stuff, the rest of the file is a posterior chain. 
From this chain, you can calculate a covariance matrix for the bias parameters 
(labelled as bias_lens--b1 etc.). 

When computing the covariance matrix, make sure to use the "weight" column, 
which provides the weight for each posterior sample.  

Once you have the covariance matrix, you can use this to define a Gaussian prior. 

In [15]:
'''#cosmological_parameters--omega_m	
cosmological_parameters--h0	
cosmological_parameters--omega_b	
cosmological_parameters--n_s	
cosmological_parameters--a_s	
cosmological_parameters--omnuh2	
shear_calibration_parameters--m1	
shear_calibration_parameters--m2	
shear_calibration_parameters--m3	
shear_calibration_parameters--m4	
wl_photoz_errors--bias_1	
wl_photoz_errors--bias_2	
wl_photoz_errors--bias_3	
wl_photoz_errors--bias_4	
lens_photoz_errors--bias_1	
lens_photoz_errors--bias_2	
lens_photoz_errors--bias_3	
lens_photoz_errors--bias_4	
lens_photoz_errors--width_1	
lens_photoz_errors--width_2	
lens_photoz_errors--width_3	
lens_photoz_errors--width_4	
22:bias_lens--b1	 ** z: 0.2 -- 0.4 
23:bias_lens--b2	 ** z: 0.4 -- 0.55
24:bias_lens--b3	 ** z: 0.55-- 0.7
25:bias_lens--b4	 ** z: 0.7 -- 0.85
intrinsic_alignment_parameters--a1	
intrinsic_alignment_parameters--a2	
intrinsic_alignment_parameters--alpha1	
intrinsic_alignment_parameters--alpha2	
intrinsic_alignment_parameters--bias_ta	
COSMOLOGICAL_PARAMETERS--SIGMA_8	
COSMOLOGICAL_PARAMETERS--SIGMA_12	
DATA_VECTOR--2PT_CHI2	
prior	 **
like	 **
post	 **
weight **
'''

'#cosmological_parameters--omega_m\t\ncosmological_parameters--h0\t\ncosmological_parameters--omega_b\t\ncosmological_parameters--n_s\t\ncosmological_parameters--a_s\t\ncosmological_parameters--omnuh2\t\nshear_calibration_parameters--m1\t\nshear_calibration_parameters--m2\t\nshear_calibration_parameters--m3\t\nshear_calibration_parameters--m4\t\nwl_photoz_errors--bias_1\t\nwl_photoz_errors--bias_2\t\nwl_photoz_errors--bias_3\t\nwl_photoz_errors--bias_4\t\nlens_photoz_errors--bias_1\t\nlens_photoz_errors--bias_2\t\nlens_photoz_errors--bias_3\t\nlens_photoz_errors--bias_4\t\nlens_photoz_errors--width_1\t\nlens_photoz_errors--width_2\t\nlens_photoz_errors--width_3\t\nlens_photoz_errors--width_4\t\n22:bias_lens--b1\t ** z: 0.2 -- 0.4 \n23:bias_lens--b2\t ** z: 0.4 -- 0.55\n24:bias_lens--b3\t ** z: 0.55-- 0.7\n25:bias_lens--b4\t ** z: 0.7 -- 0.85\nintrinsic_alignment_parameters--a1\t\nintrinsic_alignment_parameters--a2\t\nintrinsic_alignment_parameters--alpha1\t\nintrinsic_alignment_paramet

In [16]:
'''22:bias_lens--b1	 ** z: 0.2 -- 0.4 
23:bias_lens--b2	 ** z: 0.4 -- 0.55
24:bias_lens--b3	 ** z: 0.55-- 0.7
25:bias_lens--b4	 ** z: 0.7 -- 0.85
37:weights
'''

data = np.loadtxt('5)data/chain_3x2pt_lcdm_SR_maglim.txt')
print(data.shape)
print(data)
bias = data[:,22:26]
print(bias.shape)
weights = data[:,37]
print(weights.shape)

(22619, 38)
[[ 3.79849364e-01  8.01906887e-01  5.43648886e-02 ... -9.60254248e+05
  -9.60191007e+05  0.00000000e+00]
 [ 3.25558138e-01  8.40476018e-01  3.29467539e-02 ... -9.38138083e+05
  -9.38076828e+05  0.00000000e+00]
 [ 2.89650909e-01  6.89025040e-01  5.66132547e-02 ... -9.24304075e+05
  -9.24237975e+05  0.00000000e+00]
 ...
 [ 3.73624337e-01  5.73931900e-01  4.68005339e-02 ...  5.80828380e+03
   5.86166583e+03  7.51098863e-05]
 [ 3.37257773e-01  6.86988949e-01  4.16535260e-02 ...  5.80835578e+03
   5.86413287e+03  8.07156234e-05]
 [ 3.55223638e-01  6.00607801e-01  4.45173197e-02 ...  5.80888910e+03
   5.85964846e+03  1.37587212e-04]]
(22619, 4)
(22619,)


In [17]:
# The background average 
print(bias.shape)
average = np.average(bias.T,axis=1,weights =  weights)
print(average.shape)
print(average)
print(weights.shape)
print(weights)

np.save('5)dataOut/bgAverage.npy',average)

(22619, 4)
(4,)
[1.48784781 1.68593073 1.90861472 1.79274429]
(22619,)
[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 7.51098863e-05
 8.07156234e-05 1.37587212e-04]


In [18]:
# The background Covariance matrix 
print(bias.shape)
samples=bias.T#.flatten()
print(samples.shape)
cov = np.cov(samples,aweights=weights )
print(cov)
print(cov.shape)
np.save('5)dataOut/bgCovMatrix.npy',cov)

(22619, 4)
(4, 22619)
[[0.00952257 0.0078341  0.00871456 0.00845201]
 [0.0078341  0.01234342 0.00953957 0.00911681]
 [0.00871456 0.00953957 0.01370079 0.01050783]
 [0.00845201 0.00911681 0.01050783 0.01267521]]
(4, 4)
